In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Load the dataset
data = pd.DataFrame({
    'user_name': ['fatimah', 'faheem', 'faheem', 'naeemah', 'naeemah', 'naeemah', 'fatimah', 'zuni', 'zuni', 'fatimah', 'zuni', 'faheem'],
    'food_item': ['Pizza', 'fries', 'Salad', 'Pizza', 'Sushi', 'Salad', 'Pizza', 'Sushi', 'burger', 'Pizza', 'burger', 'Salad'],
    'rating': [5, 3, 4, 4, 0, 5, 2, 5, 4, 4, 4, 5]
})

# Step 2: Resolve duplicate entries by averaging the ratings
data = data.groupby(['user_name', 'food_item'], as_index=False)['rating'].mean()

# Step 3: Create the user-item rating matrix
user_item_matrix = data.pivot(index='user_name', columns='food_item', values='rating').fillna(0)

# Step 4: Calculate cosine similarity between food items
food_similarity = cosine_similarity(user_item_matrix.T)
food_similarity_df = pd.DataFrame(food_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)

# Step 5: Define a function to recommend food items for a user
def recommend_foods(user_name, num_recommendations=3):
    if user_name not in user_item_matrix.index:
        return f"User '{user_name}' not found in the dataset."
    
    # Get the ratings for the specified user
    user_ratings = user_item_matrix.loc[user_name]
    
    # Identify foods the user has already rated
    rated_foods = user_ratings[user_ratings > 0].index
    
    # Predict ratings for unrated foods
    predicted_ratings = {}
    for food_item in user_item_matrix.columns:
        if food_item not in rated_foods:
            similar_foods = food_similarity_df[food_item]
            user_rated_similarities = similar_foods[rated_foods]
            user_rated_values = user_ratings[rated_foods]
            
            # Compute predicted rating using weighted average
            if user_rated_similarities.sum() > 0:
                predicted_rating = (user_rated_similarities @ user_rated_values) / user_rated_similarities.sum()
            else:
                predicted_rating = 0
            
            predicted_ratings[food_item] = predicted_rating
    
    # Sort the predicted ratings and return the top recommendations
    recommended_foods = sorted(predicted_ratings.items(), key=lambda x: x[1], reverse=True)[:num_recommendations]
    return [food for food, rating in recommended_foods]

# Step 6: Take user input and recommend food items
def main():
    print("Available Users:", list(user_item_matrix.index))
    user_name = input("Enter the User Name to get food recommendations: ").strip()
    try:
        num_recommendations = int(input("Enter the number of recommendations you want: "))
        recommended_foods = recommend_foods(user_name, num_recommendations)
        print(f"Recommended foods for {user_name}: {recommended_foods}")
    except ValueError:
        print("Invalid input. Please ensure you enter a valid number for recommendations.")

if __name__ == "__main__":
    main()


Available Users: ['faheem', 'fatimah', 'naeemah', 'zuni']
